In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_02 import *
from jax.experimental import stax, optimizers
from jax.experimental.stax import Dense, Relu

In [3]:
mpl.rcParams['image.cmap'] = 'gray'

In [4]:
x_train,y_train,x_valid,y_valid = get_data()

/home/john/anaconda3/envs/jax/lib/python3.7/site-packages/jax/lib/xla_bridge.py:114: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [5]:
n,m = x_train.shape
c = y_train.max()+1
nh = 50

In [6]:
random_init_params, net = stax.serial(Dense(50), Relu, Dense(10), Relu)

In [7]:
key = jax.random.PRNGKey(3)
_, init_params = random_init_params(key, (-1, 28*28))

In [8]:
pred = net(init_params, x_train)

### Cross entropy loss

In [9]:
def log_softmax(x): return np.log(np.exp(x)/np.sum(np.exp(pred), axis=-1, keepdims=True))

In [10]:
sm_pred = log_softmax(pred)

In [11]:
y_train[:3]

DeviceArray([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
             [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

In [12]:
sm_pred[0][5]

DeviceArray(-2.7762098, dtype=float32)

In [22]:
def nll(y, target): return -np.mean(y*target)

In [23]:
loss = nll(sm_pred, y_train)

In [24]:
loss

DeviceArray(0.23491286, dtype=float32)

In [37]:
def logsumexp(x):
    m = np.max(x, axis=-1)[0]
    return m + np.log(np.sum(np.exp(x-m), axis=-1, keepdims=True))

In [38]:
def log_softmax(x): return x - logsumexp(x)

In [39]:
nll(log_softmax(pred), y_train) - loss

DeviceArray(0., dtype=float32)